<h1>Implement the Katz Backoff model for the quadrigram model.

In [ ]:
import pickle, math, collections
from tqdm import tqdm

def load_counts(n):
    """Load pickled n-gram counts"""
    with open(f"ngram_counts_{n}gram.pkl", "rb") as f:
        return pickle.load(f)

def good_turing_discount(c, Nc):
    """Compute discounted count using Good–Turing"""
    Nc_c = Nc.get(c, 0)
    Nc_c1 = Nc.get(c + 1, 0)
    if c == 0 or Nc_c == 0 or Nc_c1 == 0:
        return c  # fallback to raw count
    return (c + 1) * (Nc_c1 / Nc_c)

# Katz Backoff
class KatzBackoff:
    def __init__(self, uni, bi, tri, quad, cutoff=5):
        self.models = [None, uni, bi, tri, quad]
        self.cutoff = cutoff
        self.N = sum(uni.values())  # total tokens

        # Nc tables: frequency of frequencies
        self.Nc = [None] + [collections.Counter(m.values()) for m in self.models[1:]]
        
        # Followers: for alpha computation
        self.followers = [None, {}, {}, {}]
        for n in range(2, 5):
            for ngram in self.models[n]:
                context, word = ngram[:-1], ngram[-1]
                self.followers[n-1].setdefault(context, set()).add(word)

        self.alpha_cache = [None, {}, {}, {}]
        self.prob_cache = {}

    def prob_seen(self, ngram):
        """Discounted prob for seen n-grams"""
        n, c = len(ngram), self.models[len(ngram)][ngram]
        context = ngram[:-1]
        denom = self.N if n == 1 else self.models[n-1].get(context, 0)
        if denom == 0: return 0
        if c > self.cutoff: return c / denom
        return good_turing_discount(c, self.Nc[n]) / denom
 
    def alpha(self, context):
        """Compute alpha(context) — backoff weight"""
        n = len(context)
        if context in self.alpha_cache[n]:
            return self.alpha_cache[n][context]

        seen = self.followers[n].get(context, set())
        num = 1 - sum(self.prob(context + (w,)) for w in seen)
        den = 1 - sum(self.prob(context[1:] + (w,)) for w in seen)
        alpha = num / den if den > 0 else 1.0
        self.alpha_cache[n][context] = alpha
        return alpha

    def prob(self, ngram):
        """Recursive Katz backoff probability"""
        if ngram in self.prob_cache:
            return self.prob_cache[ngram]

        n = len(ngram)
        if n == 1:
            prob = self.prob_seen(ngram) or 1e-9
        elif ngram in self.models[n]:
            prob = self.prob_seen(ngram)
        else:
            prob = self.alpha(ngram[:-1]) * self.prob(ngram[1:])
        
        self.prob_cache[ngram] = prob
        return prob

    def sent_logprob(self, sentence, n=4):
        tokens = ["<s>"]*(n-1) + sentence.split() + ["</s>"]
        total_logp = 0
        for i in range(n-1, len(tokens)):
            ngram = tuple(tokens[i-n+1:i+1])
            p = self.prob(ngram)
            total_logp += math.log(p if p > 0 else 1e-12)
        return total_logp, len(tokens) - (n-1)

def perplexity(model, filepath):
    total_logp, total_words = 0, 0
    with open(filepath, encoding="utf-8") as f:
        for line in tqdm(f, desc=f"Perplexity {filepath}"):
            s = line.strip()
            if not s: continue
            model.prob_cache.clear()
            lp, wc = model.sent_logprob(s)
            total_logp += lp
            total_words += wc
    return math.exp(-total_logp / total_words)

if __name__ == "__main__":
    uni, bi, tri, quad = [load_counts(i) for i in range(1,5)]
    model = KatzBackoff(uni, bi, tri, quad)

    val_ppl = perplexity(model, "validation.txt")
    print(f"Validation Perplexity: {val_ppl:.3f}")

    test_ppl = perplexity(model, "test.txt")
    print(f"Test Perplexity: {test_ppl:.3f}")

Initializing Katz Backoff Model...
Preprocessing follower sets for alpha calculation...


Processing 4-grams: 100%|██████████| 11619213/11619213 [00:52<00:00, 219453.42it/s]


Initialization complete.

Evaluating perplexity on validation.txt...


Calculating Perplexity: 100%|██████████| 1000/1000 [00:12<00:00, 80.54it/s]


Validation Set Perplexity: 4183.7198

Evaluating perplexity on test.txt...


Calculating Perplexity: 100%|██████████| 1000/1000 [00:03<00:00, 330.10it/s]

Test Set Perplexity: 4433.3625


<h1>Implement the Kneser-Ney smoothing algorithm for the quadrigram model.

In [ ]:
import pickle
import collections
import math
from tqdm import tqdm

print("Loading all n-gram count files...")
all_counts = [None] 
for n in range(1, 5):
    file_path = f"ngram_counts_{n}gram.pkl"
    try:
        with open(file_path, "rb") as f:
            counts = pickle.load(f)
            all_counts.append(counts)
        print(f"Successfully loaded {file_path}")
    except FileNotFoundError:
        print(f"Error: Could not find the file {file_path}. Please ensure it's in the correct directory.")
        exit() 

unigrams, bigrams, trigrams, quadrigrams = all_counts[1:]
DISCOUNT = 0.75

print("Performing pre-calculations for Kneser-Ney...")
preceders_of_word = collections.defaultdict(set)
for w1, w2 in bigrams:
    preceders_of_word[w2].add(w1)
total_bigram_types = len(bigrams)

followers_of_unigram = collections.defaultdict(set)
for w1, w2 in bigrams:
    followers_of_unigram[(w1,)].add(w2)

followers_of_bigram = collections.defaultdict(set)
for w1, w2, w3 in trigrams:
    followers_of_bigram[(w1, w2)].add(w3)

followers_of_trigram = collections.defaultdict(set)
for w1, w2, w3, w4 in quadrigrams:
    followers_of_trigram[(w1, w2, w3)].add(w4)

def get_kn_prob(ngram):
    if len(ngram) == 1:
        word = ngram[0]
        numerator = len(preceders_of_word.get(word, set()))
        return numerator / total_bigram_types if total_bigram_types > 0 else 0

    n = len(ngram)
    context = ngram[:-1]
    count_ngram = all_counts[n].get(ngram, 0)
    count_context = all_counts[n-1].get(context, 0)

    if count_context == 0:
        return get_kn_prob(ngram[1:])

    first_term = max(count_ngram - DISCOUNT, 0) / count_context

    if n == 4: followers = followers_of_trigram.get(context, set())
    elif n == 3: followers = followers_of_bigram.get(context, set())
    else: followers = followers_of_unigram.get(context, set())
    
    lambda_weight = (DISCOUNT / count_context) * len(followers)

    lower_order_prob = get_kn_prob(ngram[1:])

    return first_term + lambda_weight * lower_order_prob

def calculate_perplexity(filepath):
    print(f"\nEvaluating perplexity on {filepath}...")
    total_log_prob = 0
    total_words = 0
    n = 4

    with open(filepath, "r", encoding="utf-8") as f:
        for sentence in tqdm(f, desc="Calculating Perplexity"):
            if not sentence.strip(): continue
            tokens = ["<s>"] * (n - 1) + sentence.strip().split() + ["</s>"]
            
            for i in range(n - 1, len(tokens)):
                ngram = tuple(tokens[i - n + 1 : i + 1])
                prob = get_kn_prob(ngram)
                total_log_prob += math.log(prob if prob > 0 else 1e-12)
            
            total_words += len(sentence.strip().split()) + 1

    if total_words == 0: return float('inf')
    
    perplexity = math.exp(-total_log_prob / total_words)
    return perplexity

val_perplexity = calculate_perplexity("validation.txt")
print(f"Validation Set Perplexity: {val_perplexity:.4f}")

test_perplexity = calculate_perplexity("test.txt")
print(f"Test Set Perplexity: {test_perplexity:.4f}")

Loading all n-gram count files...
Successfully loaded ngram_counts_1gram.pkl
Successfully loaded ngram_counts_2gram.pkl
Successfully loaded ngram_counts_3gram.pkl
Successfully loaded ngram_counts_4gram.pkl
Performing pre-calculations for Kneser-Ney...

Evaluating perplexity on validation.txt...


Calculating Perplexity: 1000it [00:00, 1455.27it/s]


Validation Set Perplexity: 1375.6543

Evaluating perplexity on test.txt...


Calculating Perplexity: 1000it [00:00, 2841.59it/s]

Test Set Perplexity: 1552.9169


In [ ]:
import pickle
import collections
import math
from tqdm import tqdm
import csv 
print("Loading all n-gram count files...")
all_counts = [None]
for n in range(1, 5):
    file_path = f"ngram_counts_{n}gram.pkl"
    try:
        with open(file_path, "rb") as f:
            counts = pickle.load(f)
            all_counts.append(counts)
        print(f"Successfully loaded {file_path}")
    except FileNotFoundError:
        print(f"Error: Could not find the file {file_path}. Please ensure it's in the correct directory.")
        exit()

unigrams, bigrams, trigrams, quadrigrams = all_counts[1:]
DISCOUNT = 0.75

print("Performing pre-calculations for Kneser-Ney...")
preceders_of_word = collections.defaultdict(set)
for w1, w2 in bigrams:
    preceders_of_word[w2].add(w1)
total_bigram_types = len(bigrams)

followers_of_unigram = collections.defaultdict(set)
for w1, w2 in bigrams:
    followers_of_unigram[(w1,)].add(w2)

followers_of_bigram = collections.defaultdict(set)
for w1, w2, w3 in trigrams:
    followers_of_bigram[(w1, w2)].add(w3)

followers_of_trigram = collections.defaultdict(set)
for w1, w2, w3, w4 in quadrigrams:
    followers_of_trigram[(w1, w2, w3)].add(w4)

def get_kn_prob(ngram):
    """Calculates the Kneser-Ney probability of an n-gram recursively."""
    if len(ngram) == 1:
        word = ngram[0]
        numerator = len(preceders_of_word.get(word, set()))
        return numerator / total_bigram_types if total_bigram_types > 0 else 0

    n = len(ngram)
    context = ngram[:-1]
    count_ngram = all_counts[n].get(ngram, 0)
    count_context = all_counts[n-1].get(context, 0)

    if count_context == 0:
        return get_kn_prob(ngram[1:])

    first_term = max(count_ngram - DISCOUNT, 0) / count_context

    if n == 4: followers = followers_of_trigram.get(context, set())
    elif n == 3: followers = followers_of_bigram.get(context, set())
    else: followers = followers_of_unigram.get(context, set())
    lambda_weight = (DISCOUNT / count_context) * len(followers)

    lower_order_prob = get_kn_prob(ngram[1:])

    return first_term + lambda_weight * lower_order_prob

# --- MODIFIED EVALUATION FUNCTION ---
def evaluate_sentence_probabilities(filepath):
    """Calculates and saves the probability and log-prob for each sentence."""
    output_filename = filepath.replace('.txt', '_probabilities.csv')
    print(f"\nCalculating probabilities for {filepath}...")
    print(f"Results will be saved to {output_filename}")

    n = 4 # Quadrigram model

    # Open the output CSV file to write the results
    with open(filepath, "r", encoding="utf-8") as infile, \
         open(output_filename, "w", encoding="utf-8", newline='') as outfile:

        writer = csv.writer(outfile)
        # Write the header row for the CSV file
        writer.writerow(["Sentence", "Probability", "Log Probability"])

        for sentence in tqdm(infile, desc="Processing Sentences"):
            sentence = sentence.strip()
            if not sentence: continue

            # Initialize the log probability for the current sentence
            sentence_log_prob = 0.0
            
            # Pad the sentence with start and end tokens
            tokens = ["<s>"] * (n - 1) + sentence.split() + ["</s>"]

            # Calculate the log probability by summing the log probs of its n-grams
            for i in range(n - 1, len(tokens)):
                ngram = tuple(tokens[i - n + 1 : i + 1])
                prob = get_kn_prob(ngram)
                sentence_log_prob += math.log(prob if prob > 0 else 1e-12)
            
            # Convert the total log probability back to a regular probability
            # P(sentence) = exp(log(P(sentence)))
            sentence_prob = math.exp(sentence_log_prob)
            
            # Write the sentence and its calculated scores to the CSV file
            writer.writerow([sentence, f"{sentence_prob:.4e}", f"{sentence_log_prob:.4f}"])

# --- MODIFIED EXECUTION BLOCK ---
evaluate_sentence_probabilities("validation.txt")
evaluate_sentence_probabilities("test.txt")

print("\nProcessing complete.")

Loading all n-gram count files...
Successfully loaded ngram_counts_1gram.pkl
Successfully loaded ngram_counts_2gram.pkl
Successfully loaded ngram_counts_3gram.pkl
Successfully loaded ngram_counts_4gram.pkl
Performing pre-calculations for Kneser-Ney...

Calculating probabilities for validation.txt...
Results will be saved to validation_probabilities.csv


Processing Sentences: 1000it [00:00, 1597.22it/s]



Calculating probabilities for test.txt...
Results will be saved to test_probabilities.csv


Processing Sentences: 1000it [00:00, 3972.18it/s]


Processing complete.


<h1>For each of the n-gram models generate 100 sentences using<br>
a. Greedy Approach (using maximum likelihood estimation)<br>
b. Beam Search with beam size=20

In [ ]:
import pickle
import math
import heapq
import random
import numpy as np
from tqdm import tqdm

def load_ngram_counts(n):
    file_name = f"ngram_counts_{n}gram.pkl"
    try:
        with open(file_name, "rb") as f:
            return pickle.load(f)
    except FileNotFoundError:
        print(f"Error: The file {file_name} was not found.")
        exit()

class SentenceGenerator:
    def __init__(self, counts1, counts2, counts3, counts4):
        print("Initializing Sentence Generator...")
        self.unigrams = counts1
        self.bigram_map = self._preprocess_and_sort_counts(counts2)
        self.trigram_map = self._preprocess_and_sort_counts(counts3)
        self.quadrigram_map = self._preprocess_and_sort_counts(counts4)
        self.all_counts = [None, counts1, counts2, counts3, counts4]
        self.all_maps = [None, None, self.bigram_map, self.trigram_map, self.quadrigram_map]

        total = sum(self.unigrams.values())
        self.unigram_probs = [(w[0], c / total) for w, c in self.unigrams.items()]
        self.sorted_unigrams = sorted(self.unigram_probs, key=lambda x: x[1], reverse=True)
        print("Initialization complete.\n")

    def _preprocess_and_sort_counts(self, ngram_counts):
        """Creates sorted map: context → [(word, count)...]"""
        processed = {}
        for ngram, count in ngram_counts.items():
            ctx, w = ngram[:-1], ngram[-1]
            processed.setdefault(ctx, {})[w] = count
        sorted_map = {ctx: sorted(words.items(), key=lambda x: x[1], reverse=True)
                      for ctx, words in processed.items()}
        return sorted_map

    def _get_probs(self, context, top_k=None):
        """Backoff to lower n if context unseen."""
        n = len(context) + 1
        if n == 1:
            return self.sorted_unigrams

        next_words = self.all_maps[n].get(context)
        if not next_words:
            return self._get_probs(context[1:], top_k)

        if top_k:
            next_words = next_words[:top_k]
        ctx_count = self.all_counts[n - 1].get(context, 0)
        if ctx_count == 0:
            return self._get_probs(context[1:], top_k)
        return [(w, c / ctx_count) for w, c in next_words]


    # ---------- Generation methods ----------
    def generate_greedy(self, n, num_sentences=100, max_len=30):
        print(f"\nGenerating {num_sentences} Greedy {n}-gram sentences...")
        sentences = []
        for _ in tqdm(range(num_sentences), desc=f"Greedy {n}-gram"):
            context = tuple(["<s>"] * (n - 1))
            sentence = []
            for _ in range(max_len):
                probs = self._get_probs(context)
                if not probs: break
                word = probs[0][0]  # most probable
                if word == "</s>": break
                sentence.append(word)
                if n > 1: context = context[1:] + (word,)
            sentences.append(" ".join(sentence))
        return sentences

    def generate_beam(self, n, num_sentences=100, beam_width=10, max_len=30):
        print(f"\nGenerating {num_sentences} Beam Search {n}-gram sentences (k={beam_width})...")
        sentences = []
        for _ in tqdm(range(num_sentences), desc=f"Beam {n}-gram"):
            beam = [(0.0, ["<s>"] * (n - 1))]
            for _ in range(max_len):
                all_candidates = []
                for log_p, seq in beam:
                    if seq[-1] == "</s>":
                        all_candidates.append((log_p, seq))
                        continue
                    context = tuple(seq[-(n - 1):])
                    next_probs = self._get_probs(context, top_k=beam_width * 2)
                    if not next_probs:
                        continue
                    for w, p in next_probs:
                        p = max(p, 1e-12)
                        all_candidates.append((log_p + math.log(p) + random.uniform(-1e-4, 1e-4), seq + [w]))
                beam = heapq.nlargest(beam_width, all_candidates, key=lambda x: x[0])
                if all(b[1][-1] == "</s>" for b in beam):
                    break
            best_seq = max(beam, key=lambda x: x[0])[1]
            sent = " ".join([w for w in best_seq if w not in ("<s>", "</s>")])
            sentences.append(sent)
        return sentences

# ----------------- Main Execution -----------------
if __name__ == "__main__":
    unigrams = load_ngram_counts(1)
    bigrams = load_ngram_counts(2)
    trigrams = load_ngram_counts(3)
    quadrigrams = load_ngram_counts(4)

    gen = SentenceGenerator(unigrams, bigrams, trigrams, quadrigrams)

    for n in range(2, 5):
        greedy = gen.generate_greedy(n=n, num_sentences=20)
        with open(f"greedy_{n}gram.txt", "w", encoding="utf-8") as f:
            f.writelines([s + "\n" for s in greedy])

        top_p = gen.generate_top_p(n=n, num_sentences=20, p_threshold=0.9)
        with open(f"topp_{n}gram.txt", "w", encoding="utf-8") as f:
            f.writelines([s + "\n" for s in top_p])

        beam = gen.generate_beam(n=n, num_sentences=20, beam_width=10)
        with open(f"beam_{n}gram.txt", "w", encoding="utf-8") as f:
            f.writelines([s + "\n" for s in beam])

        print(f"✅ Saved {n}-gram results: greedy_{n}gram.txt | topp_{n}gram.txt | beam_{n}gram.txt")

Initializing Sentence Generator...
Initialization complete.


Generating 20 Greedy 2-gram sentences...


Greedy 2-gram: 100%|██████████| 20/20 [00:00<00:00, 91678.78it/s]



Generating 20 Top-p 2-gram sentences (p=0.9)...


Top-p 2-gram: 100%|██████████| 20/20 [00:06<00:00,  3.26it/s]



Generating 20 Beam Search 2-gram sentences (k=10)...


...


Generating 20 Top-p 4-gram sentences (p=0.9)...


Top-p 4-gram: 100%|██████████| 20/20 [00:05<00:00,  3.88it/s]



Generating 20 Beam Search 4-gram sentences (k=10)...


Beam 4-gram: 100%|██████████| 20/20 [00:20<00:00,  1.03s/it]

✅ Saved 4-gram results: greedy_4gram.txt | topp_4gram.txt | beam_4gram.txt
